In [1]:
import os
import sys
import git
import pathlib

import random

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
os.environ['CUDA_VISIBLE_DEVICES'] = '0' # use GPU
# Using GPU during inference has deterministic results (same as CPU)

PROJ_ROOT_PATH = pathlib.Path(git.Repo('.', search_parent_directories=True).working_tree_dir)
PROJ_ROOT =  str(PROJ_ROOT_PATH)
if PROJ_ROOT not in sys.path:
    sys.path.append(PROJ_ROOT)

from libs import utils, mnist32_cnn
from libs.constants import model_seeds
from libs.fitnessfns import ff_lenet_3hidden_ERRexpbitflips
from timeit import default_timer as timer

In [2]:
# set seed
this_seed = 9387
tf.random.set_seed(this_seed)
np.random.seed(this_seed)
random.seed(this_seed)
os.environ['PYTHONHASHSEED']=str(this_seed)

In [3]:
# Limit GPU growth
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [4]:
# Prepare dataset
# Combine test and train images together into one dataset
DATASET_PATH = str(pathlib.Path(PROJ_ROOT_PATH / "datasets" / "mnist.npz" ))
(train_images, train_labels), (test_images, test_labels) = mnist.load_data(path=DATASET_PATH)
train_images = train_images.astype(np.float32) / 255.0
test_images = test_images.astype(np.float32) / 255.0  

all_images =np.concatenate([train_images, test_images], axis=0)
all_labels =np.concatenate([train_labels, test_labels], axis=0)
all_images = np.expand_dims(all_images, axis=-1)

# resize the input shape , i.e. old shape: 28, new shape: 32
image_x_size = 32
image_y_size = 32
all_images = tf.image.resize(all_images, [image_x_size, image_y_size]) 

In [5]:
# Load test set
# Testing with only the last 12_80 images
test_set = (all_images[-12_80:], all_labels[-12_80:])

In [6]:
# Batchsize for evaluation
EVAL_BATCHSIZE = 128

In [7]:
def eval_error_injected_no_shuffle(model, 
                                  error_layer, 
                                  error_profile, 
                                  ERR_PARAM,
                                  test_set,
                                  batchsize):
    
    error_profile_c0=None
    error_profile_h0=None
    error_profile_h1=None
    error_profile_h2=None
    error_profile_op=None
    if error_layer == "c0":
        error_profile_c0=error_profile
    elif error_layer == "h0":
        error_profile_h0=error_profile
    elif error_layer =="h1":
        error_profile_h1=error_profile
    elif error_layer == "h2":
        error_profile_h2=error_profile
    elif error_layer == "op":
        error_profile_op=error_profile
    elif error_layer == "all":
        error_profile_c0=error_profile
        error_profile_h0=error_profile
        error_profile_h1=error_profile
        error_profile_h2=error_profile
        error_profile_op=error_profile
        
    acc_list = []
    for _ in range(3):
        accuracy = ff_lenet_3hidden_ERRexpbitflips(model,
                                                    error_profile_c0,
                                                    error_profile_h0,
                                                    error_profile_h1,
                                                    error_profile_h2,
                                                    error_profile_op,
                                                    ERR_PARAM=ERR_PARAM,
                                                    clayer0_shuffle_order=None,
                                                    hlayer0_shuffle_order=None,
                                                    hlayer1_shuffle_order=None,
                                                    hlayer2_shuffle_order=None,
                                                    oplayer_shuffle_order=None,
                                                    test_set=test_set,
                                                    batchsize=batchsize)
        acc_list.append(accuracy)
    return np.mean(acc_list)

In [8]:
from libs.constants import error_seeds
error_profile_tag_list = []
MAX_FAULT_PROB_LIST = [
                       #  1E-3, 2E-3, 5E-3, 
                       # 10E-3, 20E-3, 50E-3, 99E-3,
                       100E-3, 200E-3, 500E-3, 999E-3]
for MAX_FAULT_PROB in MAX_FAULT_PROB_LIST:
    for seed in error_seeds:
        err_tag = "LIM_" + f"{int(MAX_FAULT_PROB*1E3):02d}" + "-" +str(seed)
        error_profile_tag_list.append(err_tag)
# error_profile_tag_list

In [9]:
model_type = "mnist32-cnn_1024_256_64"
# error_profile_tag_list = ['LIM_01-2188', 'LIM_01-3987', 'LIM_05-2188', 'LIM_05-3987']
results = {}
for model_seed in  model_seeds:
    results[model_seed] = {}
    # Get model
    model_instance = model_type + "-" + str(model_seed)
    dataset, model_arch, model_config, layer_widths, seed = utils.instancename2metadata(model_instance)
    model_meta_type, model_type, model_instance = utils.metadata2instancenames(dataset, model_arch, layer_widths, seed)
    model_folder = pathlib.Path(PROJ_ROOT_PATH / "models" / model_meta_type / model_type)
    model_filename = model_instance + ".h5"
    model_file = pathlib.Path(model_folder/ model_filename)
    # Load model
    model = tf.keras.models.load_model(model_file)
    print(model_instance)
    
    # Original Accuracy
    original_accuracy = ff_lenet_3hidden_ERRexpbitflips(model,
                                                        error_profile_c0=None,
                                                        error_profile_h0=None,
                                                        error_profile_h1=None,
                                                        error_profile_h2=None,
                                                        error_profile_op=None,
                                                        ERR_PARAM=None,
                                                        clayer0_shuffle_order=None,
                                                        hlayer0_shuffle_order=None,
                                                        hlayer1_shuffle_order=None,
                                                        hlayer2_shuffle_order=None,
                                                        oplayer_shuffle_order=None,
                                                        test_set=test_set,
                                                        batchsize=EVAL_BATCHSIZE)
    print("ORIGINAL ACC: ", np.around(original_accuracy, decimals=4))
    results[model_seed]["original_acc"] = np.around(original_accuracy, decimals=4)
    
    
    # Error Injection
    for error_profile_tag in error_profile_tag_list:
        print(error_profile_tag)
        results[model_seed][error_profile_tag] = {}
        # Load error profile
        err_profile_filename = "../error_profiles/"+ error_profile_tag + ".npy"
        error_profile = np.load(err_profile_filename)

        ERR_PARAM_list = [0, 2, 3]
        # ERR_PARAM_list = [0, 1,-1, 2, 3]
        error_layer_list = ["c0", "h0", "h1", "h2", "op", "all"]
        for ERR_PARAM in ERR_PARAM_list:
            results[model_seed][error_profile_tag][ERR_PARAM]={}
            for error_layer in error_layer_list:
                eval_acc = eval_error_injected_no_shuffle(model, 
                                                          error_layer, 
                                                          error_profile, 
                                                          ERR_PARAM,
                                                          test_set,
                                                          EVAL_BATCHSIZE)
                results[model_seed][error_profile_tag][ERR_PARAM][error_layer] = np.around(eval_acc,decimals=4)
                results_filename = 'fault_injection_analysis_results-all_model_seeds--higherror.npy'
                np.save(results_filename, results) 
                print(ERR_PARAM, '\t' ,error_layer, '\t' , np.around(eval_acc,decimals=4))
            print('*'*50)    
        print('*'*50)
        print('*'*50)
        print()

mnist32-cnn_1024_256_64-1023
ORIGINAL ACC:  0.9922
LIM_100-2188
0 	 c0 	 0.9919
0 	 h0 	 0.9901
0 	 h1 	 0.9911
0 	 h2 	 0.9883
0 	 op 	 0.9917
0 	 all 	 0.9854
**************************************************
2 	 c0 	 0.9922
2 	 h0 	 0.9893
2 	 h1 	 0.9883
2 	 h2 	 0.9891
2 	 op 	 0.9755
2 	 all 	 0.9643
**************************************************
3 	 c0 	 0.9919
3 	 h0 	 0.9898
3 	 h1 	 0.9901
3 	 h2 	 0.988
3 	 op 	 0.9708
3 	 all 	 0.9589
**************************************************
**************************************************
**************************************************

LIM_100-3987
0 	 c0 	 0.9909
0 	 h0 	 0.9904
0 	 h1 	 0.9904
0 	 h2 	 0.9901
0 	 op 	 0.9917
0 	 all 	 0.9872
**************************************************
2 	 c0 	 0.9914
2 	 h0 	 0.9888
2 	 h1 	 0.9885
2 	 h2 	 0.987
2 	 op 	 0.9674
2 	 all 	 0.9549
**************************************************
3 	 c0 	 0.9917
3 	 h0 	 0.9896
3 	 h1 	 0.9885
3 	 h2 	 0.987
3 	 op 	 0.9737
3 	